In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

# ==========================================================
# 1 — USAR TU DATA DE UNA ESTACIÓN
# ==========================================================
df = df_aya_mensual_P
df['T'] = pd.to_datetime(df['T'])
df = df.sort_values("T")


# ==========================================================
# 2 — FUNCIÓN SPI (McKee)
# ==========================================================
def calcular_spi_mckee(serie, ventana):
    # 1) Acumulado
    acum = serie.rolling(ventana).sum().dropna()

    # 2) Probabilidad de cero
    q = (acum == 0).mean()

    # 3) Ajuste Gamma
    datos = acum[acum > 0]
    alpha, loc, beta = stats.gamma.fit(datos, floc=0)

    # 4) Probabilidad acumulada
    F = q + (1-q) * stats.gamma.cdf(datos, alpha, loc=0, scale=beta)

    # 5) Convertir a SPI
    spi_aya_P = stats.norm.ppf(F)

    # 6) Reconstruir serie
    resultado = np.full(len(serie), np.nan)
    resultado[acum.index] = spi_aya_P
    return resultado


# ==========================================================
# 3 — CALCULAR SPI 3, 6, 12, 24
# ==========================================================
ventanas = [3, 6]

for v in ventanas:
    df[f"SPI_{v}"] = calcular_spi_mckee(df['Prec'], v)


# ==========================================================
# 4 — GRAFICAR SPI PARA ESTA ESTACIÓN
# ==========================================================
for v in [3, 6]:

    plt.figure(figsize=(14, 6))
    spi_data = df[f"SPI_{v}"]

    colores = ['red' if x < 0 else 'blue' for x in spi_data]

    plt.bar(df['T'], spi_data, color=colores, width=25)

    plt.axhline(0, color='black', linestyle='--')
    plt.axhline(-1, color='grey', linestyle=':', alpha=0.6)
    plt.axhline(1, color='grey', linestyle=':', alpha=0.6)
    plt.axhline(-2, color='grey', linestyle=':', alpha=0.6)
    plt.axhline(2, color='grey', linestyle=':', alpha=0.6)

    plt.title(f"Estación Ayabaca (SPI-{v}) PISCO")
    plt.ylabel("SPI")
    plt.xlabel("Año")

    # Etiquetas de año
    años = pd.date_range(df['T'].min(), df['T'].max(), freq='YS')
    plt.xticks(años, [str(a.year) for a in años], rotation=90)

    # Leyenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='blue', label='Húmedo'),
        Patch(facecolor='red', label='Seco')
    ]
    plt.legend(handles=legend_elements, loc='upper right', frameon=True)

    plt.grid(alpha=0.3)
    plt.tight_layout()

    plt.savefig(f"spi_ayabaca_pisco_{v}.png", dpi=300, bbox_inches='tight')
    plt.show()


# ==========================================================
# 5 — EXPORTAR TABLA FINAL
# ==========================================================
df.to_csv("spi_aya_mensual.csv", index=False)
print("Listo. SPI para la estación Ayabaca generado.")